# Análisis de Datos Meteorológicos
Por Sergio Rasillo Flores

## Creación del dataset

In [3]:
import numpy as np

np.random.seed(42)
n_estaciones, n_dias, n_horas, n_vars = 5, 30, 24, 6
datos = np.zeros((n_estaciones, n_dias, n_horas, n_vars))

# Temperatura base por estación + variación diaria + ruido
temp_base = np.array([8, 12, 14, 10, 6]).reshape(5, 1, 1)
hora = np.arange(24).reshape(1, 1, 24)
variacion_diaria = 5 * np.sin((hora - 6) * np.pi / 12)
datos[:, :, :, 0] = temp_base + variacion_diaria + np.random.normal(0, 2, (5, 30, 24))

# Resto de variables (humedad, presión, viento, precipitación, radiación)
datos[:, :, :, 1] = np.clip(
    70 - datos[:, :, :, 0] + np.random.normal(0, 10, (5, 30, 24)), 30, 100
)
datos[:, :, :, 2] = 1013 + np.random.normal(0, 5, (5, 30, 24))
datos[:, :, :, 3] = np.abs(np.random.normal(15, 10, (5, 30, 24)))
datos[:, :, :, 4] = np.maximum(0, np.random.exponential(2, (5, 30, 24)))
radiacion_base = np.maximum(0, 400 * np.sin((hora - 6) * np.pi / 12))
datos[:, :, :, 5] = np.clip(
    radiacion_base + np.random.normal(0, 50, (5, 30, 24)), 0, 800
)

# Guardar en carpeta data/
np.save('datos-meteo/datos_meteorologicos.npy', datos)
print(f"Datos guardados: {datos.shape}")

Datos guardados: (5, 30, 24, 6)


## Bloque 1: Exploración y acceso a datos

In [4]:
import numpy as np

datos = np.load('datos-meteo/datos_meteorologicos.npy')

# Ejercicio 01. Verificación de estructura
print(f"La forma del array datos es {datos.shape}, el numero total de mediciones es de {datos.size} y su tipo es {datos.dtype}")

# Ejercicio 02. Extracción de series temporales
temperaturas_madrid_dia_15 = datos[0, 15, :, 0]
print(f"Temperaturas de Madrid el dia 15(tamaño {temperaturas_madrid_dia_15.size}):\n {temperaturas_madrid_dia_15}")

# Ejercicio 03. Comparativa entre estaciones
temperaturas_todas_estaciones_12_dia_20 = datos[ :, 20, 12, 0]
print(f"La temperatura de todas las estaciones el dia 20 a las 12:00(tamaño {temperaturas_todas_estaciones_12_dia_20.size}):\n {temperaturas_todas_estaciones_12_dia_20}")

# Ejercicio 04. Bloque de datos
datos_barcelona_primera_semana = datos[1, :7, :, :]
print(f"La forma de los datos de Barcelona durante la primera semana es de {datos_barcelona_primera_semana.shape}")

La forma del array datos es (5, 30, 24, 6), el numero total de mediciones es de 21600 y su tipo es float64
Temperaturas de Madrid el dia 15(tamaño 24):
 [ 4.03869303  6.23584869  3.45235268  5.26788954  6.88028798  5.90346383
  8.44818496  9.31928003 10.6953522   9.98951434 12.37914737 13.82562571
 15.90228722 14.74817078 16.63649193 10.00083878 12.24464127  9.66077924
 12.37960587  5.0893082   3.82055632  3.2656808  -0.57791847  2.11886083]
La temperatura de todas las estaciones el dia 20 a las 12:00(tamaño 5):
 [13.15473662 15.56318547 18.06937909 16.02158418 13.9018551 ]
La forma de los datos de Barcelona durante la primera semana es de (7, 24, 6)


## Bloque 2: Estadística descritiva

In [ ]:
# Ejercicio 05. Temperatura media por estación
temperatura_media_por_estacion_mes = datos[ :, :, :, 0].mean(axis=(1, 2))
estacion_mas_calida = np.argmax(temperatura_media_por_estacion_mes)
estacion_mas_fria = np.argmin(temperatura_media_por_estacion_mes)
print(f"La temperatura media de todo el mes para cada estación es(tamaño {temperatura_media_por_estacion_mes.size}):\n {temperatura_media_por_estacion_mes}")
nombres_estaciones = ['Madrid', 'Barcelona', 'Valencia', 'Sevilla', 'Bilbao']
print(f"La estación más cálida es {nombres_estaciones[estacion_mas_calida]} y la más fría es {nombres_estaciones[estacion_mas_fria]}")

# Ejercicio 06. Perfil horario medio
temperatura_media_por_hora_dia = datos[ :, :, :, 0].mean(axis=(0,1))
print(f"El perfil térmico diario típico es(tamaño {temperatura_media_por_hora_dia.size}):\n {temperatura_media_por_hora_dia}")

# Ejercicio 07. Variabilidad climática
std_temperatura_por_estacion = datos[ :, :, :, 0].std(axis=(1,2))
estacion_mayor_variablidad = np.argmax(std_temperatura_por_estacion)
print(f"La desviación estandar de cada estación es(tamaño {std_temperatura_por_estacion.size}):\n {std_temperatura_por_estacion}")
print(f"La estación con mayor variablidad es la de {nombres_estaciones[estacion_mayor_variablidad]}")

# Ejercicio 08. Extremos meteorológicos
temperaturas = datos[ :, :, :, 0]
temperatura_maxima = np.argmax(temperaturas)
temperatura_minima = np.argmin(temperaturas)
estacion_max, dia_max, hora_max = np.unravel_index(temperatura_maxima, (5, 30, 24))
estacion_min, dia_min, hora_min = np.unravel_index(temperatura_minima, (5, 30, 24))
temp_max = temperaturas[estacion_max, dia_max, hora_max]
temp_min = temperaturas[estacion_min, dia_min, hora_min]
print(f"La temperatura máxima es de {temp_max:.2f} y fué en la estación {nombres_estaciones[estacion_max]}, el día {dia_max} a las {hora_max}")
print(f"La temperatura mínima es de {temp_min:.2f} y fué en la estación {nombres_estaciones[estacion_min]}, el día {dia_min} a las {hora_min}")

La temperatura media de todo el mes para cada estación es(tamaño 5):
 [ 7.97979927 12.17957761 14.05481385 10.05142752  6.00037658]
La estación más cálida es Valencia y la más fría es Bilbao
El perfil térmico diario típico es(tamaño 24):
 [ 5.03030346  5.15957045  5.47876169  6.51047736  7.64368656  9.00376843
 10.22693188 11.22172575 12.54605061 13.54669175 14.12382854 14.95674594
 15.18853598 14.8724688  14.36788922 13.68001046 12.35216023 11.69066635
 10.17147117  8.84390555  7.55498809  6.14391655  5.63976116  5.32245924]
La desviación estandar de cada estación es(tamaño 5):
 [4.1079113  4.01935776 4.11146254 4.05992189 4.09005996]
La estación con mayor variablidad es la de Valencia
La temperatura máxima es de 25.11 y fué en la estación Valencia, el día 21 a las 13
La temperatura mínima es de -4.19 y fué en la estación Bilbao, el día 23 a las 22


## Bloque 3: Filtrado y selección condicional

In [6]:
# Ejercicio 9. Detección de heladas
temperaturas_inferiores_a_0 = np.sum(temperaturas < 0)
porcentaje_del_total_temps_inf = temperaturas_inferiores_a_0 / temperaturas.size * 100
print(f"Ha habido un total de {temperaturas_inferiores_a_0} heladas y el porcentaje total de mediciones inferiores a 0 es de {porcentaje_del_total_temps_inf:.2f}%")

# Ejercicio 10. Días de lluvia significativa
precip_diaria = datos[:, :, :, 4].sum(axis=2)  # suma las 24 horas
estaciones_supera_prec, dias_supera_prec = np.where(precip_diaria > 10)
for est, dia in zip(estaciones_supera_prec, dias_supera_prec) :
    print(f"El día {dia}, la estación {nombres_estaciones[est]} superó los 10 mm de precipitación")

# Ejercicio 11. Condiciones de confort
condiciones_confort = np.sum((datos[:, :, :, 0] >= 18) & 
                              (datos[:, :, :, 0] <= 24) & 
                              (datos[:, :, :, 1] >= 40) & 
                              (datos[:, :, :, 1] <= 60) & 
                              (datos[:, :, :, 3] < 20))
porcentaje_mediciones_cumpliendo_condiciones = (condiciones_confort / datos[ :, :, :, 0].size) * 100
print(f"El porcentaje de mediciones que cumplen las condiciones de confort es de {porcentaje_mediciones_cumpliendo_condiciones:.2f}%")

Ha habido un total de 62 heladas y el porcentaje total de mediciones inferiores a 0 es de 1.72%
El día 0, la estación Madrid superó los 10 mm de precipitación
El día 1, la estación Madrid superó los 10 mm de precipitación
El día 2, la estación Madrid superó los 10 mm de precipitación
El día 3, la estación Madrid superó los 10 mm de precipitación
El día 4, la estación Madrid superó los 10 mm de precipitación
El día 5, la estación Madrid superó los 10 mm de precipitación
El día 6, la estación Madrid superó los 10 mm de precipitación
El día 7, la estación Madrid superó los 10 mm de precipitación
El día 8, la estación Madrid superó los 10 mm de precipitación
El día 9, la estación Madrid superó los 10 mm de precipitación
El día 10, la estación Madrid superó los 10 mm de precipitación
El día 11, la estación Madrid superó los 10 mm de precipitación
El día 12, la estación Madrid superó los 10 mm de precipitación
El día 13, la estación Madrid superó los 10 mm de precipitación
El día 14, la esta

## Bloque 4: Operaciones avanzadas

In [8]:
# Ejercicio 12. Normalización de datos
min = datos.min(axis=(0, 1, 2), keepdims=True)
max = datos.max(axis=(0, 1, 2), keepdims=True)

x_norm = (datos - min) / (max - min)
print(f"Datos normalizados con forma {x_norm.shape}")

# Ejercicio 13. Anomalías térmicas
temperaturas_medias_por_estacion_re = np.reshape(temperatura_media_por_estacion_mes, (5, 1, 1))
anomalias_termicas = temperaturas - temperaturas_medias_por_estacion_re

mayor_anomalia_positiva = np.max(anomalias_termicas)
print(f"La mayor anomalia positiva es la de {mayor_anomalia_positiva:.2f}°C")

# Ejercicio 14. Correlación temperatura-humedad
datos_planos = datos.reshape(-1, 6)
correlacion_temp_humedad = np.corrcoef(datos_planos, rowvar=False)
print(f"La correlación entre la temperatura y la humedad es de {correlacion_temp_humedad[0, 1]:.2f} por lo que tiene sentido físico")

# Ejercicio 15. Potencial de energía solar
radiaciones = datos[ :, :, :, 5]
radiacion_diaria = radiaciones.sum(axis=(2))
radiacion_total_est = radiacion_diaria.sum(axis=1)
estacion_mayor_potencial_solar = np.argmax(radiacion_total_est)
print(f"La estación con mayor potencial solar es {nombres_estaciones[estacion_mayor_potencial_solar]}")

radiacion_media_diaria = radiacion_diaria.mean(axis=1)
ranking_indices = np.argsort(radiacion_media_diaria)[::-1]

print("\nRanking de estaciones por energía solar media diaria:")
for i, idx in enumerate(ranking_indices, 1):
    print(f"{i}. {nombres_estaciones[idx]}: {radiacion_media_diaria[idx]:.2f} W/m²")

Datos normalizados con forma (5, 30, 24, 6)
La mayor anomalia positiva es la de 11.39°C
La correlación entre la temperatura y la humedad es de -0.44 por lo que tiene sentido físico
La estación con mayor potencial solar es Madrid

Ranking de estaciones por energía solar media diaria:
1. Madrid: 3333.65 W/m²
2. Valencia: 3305.53 W/m²
3. Barcelona: 3305.02 W/m²
4. Sevilla: 3291.87 W/m²
5. Bilbao: 3279.58 W/m²


## Bloque 5: Ejercicio integrador

In [ ]:
# Ejercicio 16